# [Module 5.1] Inferencde Pipeline 모델 생성

## Inference Pipeline <a class="anchor" id="pipeline_setup"></a>

아래 그림과 같이 위에서 생성한 전처리, 알고리즘 학습, 후처리의 세가지 모델을 가지고 1개의 단일 모델을 만들어 Inference Pipleline을 생성 합니다. <br>
**입력 데이타 가공이 없이 실제 데이타가 입력이 되면, 1개의 단일 모델을 통해서 최종적으로 예측 결과인 True, False의 결과 값이 제공 됩니다.**

![Inference-pipeline](img/Fig2.2.inference_pipeline.png)


**Machine Learning Model Pipeline (Inference Pipeline)는 create_model() 를 호출하여 만들 수 있습니다.** <br>
예를 들어 여기서는 the fitted Scikit-learn inference model, the fitted Xgboost model and the psotprocessing model 의 세가지 모델을 가지고 만듦니다.

아래는 세개 모델을 생성함. 전처리, 후처리 모델 생성시에는 환경 변수를 제공 함

In [1]:
import sagemaker
import pandas as pd
import numpy as np
import os
import time
import json
import boto3
from time import strftime, gmtime

In [2]:
role = sagemaker.get_execution_role()

In [3]:
%store -r

Unable to restore variable 's3_input_validation', ignoring (use %store -d to forget!)
The error was: <class 'KeyError'>
Unable to restore variable 's3_input_test', ignoring (use %store -d to forget!)
The error was: <class 'KeyError'>
Unable to restore variable 's3_input_train', ignoring (use %store -d to forget!)
The error was: <class 'KeyError'>


In [9]:
# %store

In [5]:
# %store preprocessor_enable_cloudwatch_metrics 
# %store preprocessor_uploaded_code_s3_prefix 
# %store preprocessor_container_log_level 
# %store preprocessor_region 
# %store preprocessor_uploaded_code_script_name 


In [6]:
timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_name = 'churn-inference-pipeline-' + timestamp_prefix
client = boto3.client('sagemaker')
response = client.create_model(
    ModelName=model_name,
    Containers=[
        {
            'Image': preprocessor_image_name,
            'ModelDataUrl': preprocessor_model_data,
            'Environment': {
                    "SAGEMAKER_SUBMIT_DIRECTORY": preprocessor_uploaded_code_s3_prefix,
                    "TRANSFORM_MODE": "feature-transform",
                    "SAGEMAKER_PROGRAM": preprocessor_uploaded_code_script_name
                }
        },
        {
            'Image': pca_image_name,
            'ModelDataUrl': pca_model_data,
            'Environment': {}
        },
        
        {
            'Image': xgb_image_name,
            'ModelDataUrl': xgb_model_data,
            "Environment": {}
        },
        {
            'Image': preprocessor_image_name,
            'ModelDataUrl': preprocessor_model_data,
            'Environment': {
                    "SAGEMAKER_SUBMIT_DIRECTORY": preprocessor_uploaded_code_s3_prefix,
                    "TRANSFORM_MODE": "inverse-label-transform",
                    "SAGEMAKER_PROGRAM": preprocessor_uploaded_code_script_name
                
                }
        },
    ],
    ExecutionRoleArn = role,
)


In [7]:
inference_pipeline_model_name = model_name

```
{
  "Version": "2008-10-17",
  "Statement": [
    {
      "Sid": "allowSageMakerToPull",
      "Effect": "Allow",
      "Principal": {
        "Service": "sagemaker.amazonaws.com"
      },
      "Action": [
        "ecr:BatchCheckLayerAvailability",
        "ecr:BatchGetImage",
        "ecr:GetDownloadUrlForLayer"
      ]
    }
  ]
}
```

In [8]:
%store inference_pipeline_model_name

Stored 'inference_pipeline_model_name' (str)
